In [1]:

import ssl
import pandas as pd
import wandb

ssl._create_default_https_context = ssl._create_unverified_context
import os
from torch.utils.data import DataLoader
from torch import nn
from torchvision.io import read_image
from torchvision.transforms import v2
import torch
import torchvision.transforms as T
import json
import urllib
import requests
from PIL import Image
from io import BytesIO
import numpy as np
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from torchvision.io import read_image
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer, SnowballStemmer
import nltk

try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context

#nltk.download()  # uncomment this line if you execute this notebook the first time, afterwards you can comment it again

nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('omw-1.4')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/blackbook/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/blackbook/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /Users/blackbook/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/blackbook/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

# Configure notebook parameters

In [2]:
dataset_base_path = '../../datasets/imageBasedModel/zalando'

In [3]:
dataset_base_path = '../../datasets'
images_dir = f"{dataset_base_path}/zalando-woman-outfit-dataset-raw"
output_dir_base = f'{dataset_base_path}/imageBasedModel/zalando/woman_outfits'

In [4]:
df = pd.read_csv(
    f'{dataset_base_path}/zalando-woman-outfit-dataset-raw/woman_outfit_clothes_with_img_file_path_and_color.csv'
)
df

,Unnamed: 0.1,Unnamed: 0,outfit_url,clothingItemName,clothingItemBrand,clothingItemDetailUrl,clothingItemPreviewImageUrl,id,clothingItemColor,image_path
0,0,0,https://www.zalando.ch/outfits/5Bay-ErCSwa/,PADDED VEST - Weste - black,Puma,https://www.zalando.ch/puma-ess-trainingsjacke...,https://img01.ztat.net/article/spp-media-p1/f0...,5Bay-ErCSwa,black,woman_outfits\5Bay-ErCSwa\0.jpg
1,1,1,https://www.zalando.ch/outfits/5Bay-ErCSwa/,SLFMY PERFECT TEE BOX CUT - T-Shirt basic - br...,Selected Femme,https://www.zalando.ch/selected-femme-sfmy-t-s...,https://img01.ztat.net/article/spp-media-p1/3b...,5Bay-ErCSwa,NaN,woman_outfits\5Bay-ErCSwa\1.jpg
2,2,2,https://www.zalando.ch/outfits/5Bay-ErCSwa/,MOM A LINE SHORT - Jeans Shorts - flash black,Levi's®,https://www.zalando.ch/levisr-mom-a-line-short...,https://img01.ztat.net/article/spp-media-p1/eb...,5Bay-ErCSwa,flash black,woman_outfits\5Bay-ErCSwa\2.jpg
3,3,3,https://www.zalando.ch/outfits/5Bay-ErCSwa/,530 UNISEX - Sneaker low - white,New Balance,https://www.zalando.ch/new-balance-mr530-sneak...,https://img01.ztat.net/article/spp-media-p1/7f...,5Bay-ErCSwa,white,woman_outfits\5Bay-ErCSwa\3.jpg
4,4,4,https://www.zalando.ch/outfits/5Bay-ErCSwa/,Sonnenbrille - brown,Swarovski,https://www.zalando.ch/swarovski-sonnenbrille-...,https://img01.ztat.net/article/spp-media-p1/03...,5Bay-ErCSwa,brown,woman_outfits\5Bay-ErCSwa\4.jpg
...,...,...,...,...,...,...,...,...,...,...
24546,24546,24546,https://www.zalando.ch/outfits/FOmLRaT0Rnu/,SLFEMMA LOAFER - Slipper - black,Selected Femme,https://www.zalando.ch/selected-femme-schnuere...,https://img01.ztat.net/article/spp-media-p1/c4...,FOmLRaT0Rnu,black,woman_outfits\FOmLRaT0Rnu\0.jpg
24547,24547,24547,https://www.zalando.ch/outfits/FOmLRaT0Rnu/,TABBY SHOULDER - Handtasche - black,Coach,https://www.zalando.ch/coach-polished-pebble-s...,https://img01.ztat.net/article/spp-media-p1/89...,FOmLRaT0Rnu,black,woman_outfits\FOmLRaT0Rnu\1.jpg
24548,24548,24548,https://www.zalando.ch/outfits/IMi9cMhhTSm/,High Heel Pumps - black,Even&Odd,https://www.zalando.ch/evenandodd-pumps-black-...,https://img01.ztat.net/article/spp-media-p1/f2...,IMi9cMhhTSm,black,woman_outfits\IMi9cMhhTSm\0.jpg
24549,24549,24549,https://www.zalando.ch/outfits/tUljaAGSTG6/,BASIC HOODED JACKET - Daunenjacke - black,Tommy Jeans,https://www.zalando.ch/tommy-jeans-basic-hoode...,https://img01.ztat.net/article/spp-media-p1/0f...,tUljaAGSTG6,black,woman_outfits\tUljaAGSTG6\2.jpg


In [5]:
df["clothingItemName"] = df["clothingItemName"].apply(lambda x: x.lower())
df

,Unnamed: 0.1,Unnamed: 0,outfit_url,clothingItemName,clothingItemBrand,clothingItemDetailUrl,clothingItemPreviewImageUrl,id,clothingItemColor,image_path
0,0,0,https://www.zalando.ch/outfits/5Bay-ErCSwa/,padded vest - weste - black,Puma,https://www.zalando.ch/puma-ess-trainingsjacke...,https://img01.ztat.net/article/spp-media-p1/f0...,5Bay-ErCSwa,black,woman_outfits\5Bay-ErCSwa\0.jpg
1,1,1,https://www.zalando.ch/outfits/5Bay-ErCSwa/,slfmy perfect tee box cut - t-shirt basic - br...,Selected Femme,https://www.zalando.ch/selected-femme-sfmy-t-s...,https://img01.ztat.net/article/spp-media-p1/3b...,5Bay-ErCSwa,NaN,woman_outfits\5Bay-ErCSwa\1.jpg
2,2,2,https://www.zalando.ch/outfits/5Bay-ErCSwa/,mom a line short - jeans shorts - flash black,Levi's®,https://www.zalando.ch/levisr-mom-a-line-short...,https://img01.ztat.net/article/spp-media-p1/eb...,5Bay-ErCSwa,flash black,woman_outfits\5Bay-ErCSwa\2.jpg
3,3,3,https://www.zalando.ch/outfits/5Bay-ErCSwa/,530 unisex - sneaker low - white,New Balance,https://www.zalando.ch/new-balance-mr530-sneak...,https://img01.ztat.net/article/spp-media-p1/7f...,5Bay-ErCSwa,white,woman_outfits\5Bay-ErCSwa\3.jpg
4,4,4,https://www.zalando.ch/outfits/5Bay-ErCSwa/,sonnenbrille - brown,Swarovski,https://www.zalando.ch/swarovski-sonnenbrille-...,https://img01.ztat.net/article/spp-media-p1/03...,5Bay-ErCSwa,brown,woman_outfits\5Bay-ErCSwa\4.jpg
...,...,...,...,...,...,...,...,...,...,...
24546,24546,24546,https://www.zalando.ch/outfits/FOmLRaT0Rnu/,slfemma loafer - slipper - black,Selected Femme,https://www.zalando.ch/selected-femme-schnuere...,https://img01.ztat.net/article/spp-media-p1/c4...,FOmLRaT0Rnu,black,woman_outfits\FOmLRaT0Rnu\0.jpg
24547,24547,24547,https://www.zalando.ch/outfits/FOmLRaT0Rnu/,tabby shoulder - handtasche - black,Coach,https://www.zalando.ch/coach-polished-pebble-s...,https://img01.ztat.net/article/spp-media-p1/89...,FOmLRaT0Rnu,black,woman_outfits\FOmLRaT0Rnu\1.jpg
24548,24548,24548,https://www.zalando.ch/outfits/IMi9cMhhTSm/,high heel pumps - black,Even&Odd,https://www.zalando.ch/evenandodd-pumps-black-...,https://img01.ztat.net/article/spp-media-p1/f2...,IMi9cMhhTSm,black,woman_outfits\IMi9cMhhTSm\0.jpg
24549,24549,24549,https://www.zalando.ch/outfits/tUljaAGSTG6/,basic hooded jacket - daunenjacke - black,Tommy Jeans,https://www.zalando.ch/tommy-jeans-basic-hoode...,https://img01.ztat.net/article/spp-media-p1/0f...,tUljaAGSTG6,black,woman_outfits\tUljaAGSTG6\2.jpg


In [6]:
df["fistToken"] = df["clothingItemName"].apply(lambda x: x.split(' - ')[0].lower())
df["secondToken"] = df["clothingItemName"].apply(lambda x: x.split(' - ')[1].lower())
df

,Unnamed: 0.1,Unnamed: 0,outfit_url,clothingItemName,clothingItemBrand,clothingItemDetailUrl,clothingItemPreviewImageUrl,id,clothingItemColor,image_path,fistToken,secondToken
0,0,0,https://www.zalando.ch/outfits/5Bay-ErCSwa/,padded vest - weste - black,Puma,https://www.zalando.ch/puma-ess-trainingsjacke...,https://img01.ztat.net/article/spp-media-p1/f0...,5Bay-ErCSwa,black,woman_outfits\5Bay-ErCSwa\0.jpg,padded vest,weste
1,1,1,https://www.zalando.ch/outfits/5Bay-ErCSwa/,slfmy perfect tee box cut - t-shirt basic - br...,Selected Femme,https://www.zalando.ch/selected-femme-sfmy-t-s...,https://img01.ztat.net/article/spp-media-p1/3b...,5Bay-ErCSwa,NaN,woman_outfits\5Bay-ErCSwa\1.jpg,slfmy perfect tee box cut,t-shirt basic
2,2,2,https://www.zalando.ch/outfits/5Bay-ErCSwa/,mom a line short - jeans shorts - flash black,Levi's®,https://www.zalando.ch/levisr-mom-a-line-short...,https://img01.ztat.net/article/spp-media-p1/eb...,5Bay-ErCSwa,flash black,woman_outfits\5Bay-ErCSwa\2.jpg,mom a line short,jeans shorts
3,3,3,https://www.zalando.ch/outfits/5Bay-ErCSwa/,530 unisex - sneaker low - white,New Balance,https://www.zalando.ch/new-balance-mr530-sneak...,https://img01.ztat.net/article/spp-media-p1/7f...,5Bay-ErCSwa,white,woman_outfits\5Bay-ErCSwa\3.jpg,530 unisex,sneaker low
4,4,4,https://www.zalando.ch/outfits/5Bay-ErCSwa/,sonnenbrille - brown,Swarovski,https://www.zalando.ch/swarovski-sonnenbrille-...,https://img01.ztat.net/article/spp-media-p1/03...,5Bay-ErCSwa,brown,woman_outfits\5Bay-ErCSwa\4.jpg,sonnenbrille,brown
...,...,...,...,...,...,...,...,...,...,...,...,...
24546,24546,24546,https://www.zalando.ch/outfits/FOmLRaT0Rnu/,slfemma loafer - slipper - black,Selected Femme,https://www.zalando.ch/selected-femme-schnuere...,https://img01.ztat.net/article/spp-media-p1/c4...,FOmLRaT0Rnu,black,woman_outfits\FOmLRaT0Rnu\0.jpg,slfemma loafer,slipper
24547,24547,24547,https://www.zalando.ch/outfits/FOmLRaT0Rnu/,tabby shoulder - handtasche - black,Coach,https://www.zalando.ch/coach-polished-pebble-s...,https://img01.ztat.net/article/spp-media-p1/89...,FOmLRaT0Rnu,black,woman_outfits\FOmLRaT0Rnu\1.jpg,tabby shoulder,handtasche
24548,24548,24548,https://www.zalando.ch/outfits/IMi9cMhhTSm/,high heel pumps - black,Even&Odd,https://www.zalando.ch/evenandodd-pumps-black-...,https://img01.ztat.net/article/spp-media-p1/f2...,IMi9cMhhTSm,black,woman_outfits\IMi9cMhhTSm\0.jpg,high heel pumps,black
24549,24549,24549,https://www.zalando.ch/outfits/tUljaAGSTG6/,basic hooded jacket - daunenjacke - black,Tommy Jeans,https://www.zalando.ch/tommy-jeans-basic-hoode...,https://img01.ztat.net/article/spp-media-p1/0f...,tUljaAGSTG6,black,woman_outfits\tUljaAGSTG6\2.jpg,basic hooded jacket,daunenjacke


In [7]:
stemmer = SnowballStemmer("english")
stop_words = set(stopwords.words('english'))


def lemmatize_stemming(text):
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))


def normalize(text):
    tokens = nltk.word_tokenize(text)
    # result = [lemmatize_stemming(token) for token in tokens
    #           if token not in stop_words and len(token) > 3]
    # return result
    return tokens


df['firstPartToken'] = df["fistToken"].apply(lambda x: normalize(x))
df['secondPartToken'] = df['secondToken'].apply(lambda x: normalize(x))

In [8]:
df.explode('firstPartToken')['firstPartToken'].value_counts()

firstPartToken
unisex          2800
pack             826
bag              564
handtasche       502
sonnenbrille     454
                ... 
bren               1
viviga             1
jaclorenzo         1
nmcosmic           1
pcmully            1
Name: count, Length: 7170, dtype: int64

In [9]:
df.explode('secondPartToken')['secondPartToken'].value_counts()

secondPartToken
sneaker             1943
black               1751
low                 1645
handtasche          1402
sonnenbrille        1103
                    ... 
neonyellow/black       1
black/black/grey       1
coral                  1
uni                    1
bedruckt               1
Name: count, Length: 1033, dtype: int64

In [10]:
from src.clothing_item_types import get_clothing_item_type

df['wearType'] = df["clothingItemName"].apply(lambda x: get_clothing_item_type(x))
df

,Unnamed: 0.1,Unnamed: 0,outfit_url,clothingItemName,clothingItemBrand,clothingItemDetailUrl,clothingItemPreviewImageUrl,id,clothingItemColor,image_path,fistToken,secondToken,firstPartToken,secondPartToken,wearType
0,0,0,https://www.zalando.ch/outfits/5Bay-ErCSwa/,padded vest - weste - black,Puma,https://www.zalando.ch/puma-ess-trainingsjacke...,https://img01.ztat.net/article/spp-media-p1/f0...,5Bay-ErCSwa,black,woman_outfits\5Bay-ErCSwa\0.jpg,padded vest,weste,"[padded, vest]",[weste],2.0
1,1,1,https://www.zalando.ch/outfits/5Bay-ErCSwa/,slfmy perfect tee box cut - t-shirt basic - br...,Selected Femme,https://www.zalando.ch/selected-femme-sfmy-t-s...,https://img01.ztat.net/article/spp-media-p1/3b...,5Bay-ErCSwa,NaN,woman_outfits\5Bay-ErCSwa\1.jpg,slfmy perfect tee box cut,t-shirt basic,"[slfmy, perfect, tee, box, cut]","[t-shirt, basic]",2.0
2,2,2,https://www.zalando.ch/outfits/5Bay-ErCSwa/,mom a line short - jeans shorts - flash black,Levi's®,https://www.zalando.ch/levisr-mom-a-line-short...,https://img01.ztat.net/article/spp-media-p1/eb...,5Bay-ErCSwa,flash black,woman_outfits\5Bay-ErCSwa\2.jpg,mom a line short,jeans shorts,"[mom, a, line, short]","[jeans, shorts]",4.0
3,3,3,https://www.zalando.ch/outfits/5Bay-ErCSwa/,530 unisex - sneaker low - white,New Balance,https://www.zalando.ch/new-balance-mr530-sneak...,https://img01.ztat.net/article/spp-media-p1/7f...,5Bay-ErCSwa,white,woman_outfits\5Bay-ErCSwa\3.jpg,530 unisex,sneaker low,"[530, unisex]","[sneaker, low]",5.0
4,4,4,https://www.zalando.ch/outfits/5Bay-ErCSwa/,sonnenbrille - brown,Swarovski,https://www.zalando.ch/swarovski-sonnenbrille-...,https://img01.ztat.net/article/spp-media-p1/03...,5Bay-ErCSwa,brown,woman_outfits\5Bay-ErCSwa\4.jpg,sonnenbrille,brown,[sonnenbrille],[brown],1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24546,24546,24546,https://www.zalando.ch/outfits/FOmLRaT0Rnu/,slfemma loafer - slipper - black,Selected Femme,https://www.zalando.ch/selected-femme-schnuere...,https://img01.ztat.net/article/spp-media-p1/c4...,FOmLRaT0Rnu,black,woman_outfits\FOmLRaT0Rnu\0.jpg,slfemma loafer,slipper,"[slfemma, loafer]",[slipper],5.0
24547,24547,24547,https://www.zalando.ch/outfits/FOmLRaT0Rnu/,tabby shoulder - handtasche - black,Coach,https://www.zalando.ch/coach-polished-pebble-s...,https://img01.ztat.net/article/spp-media-p1/89...,FOmLRaT0Rnu,black,woman_outfits\FOmLRaT0Rnu\1.jpg,tabby shoulder,handtasche,"[tabby, shoulder]",[handtasche],1.0
24548,24548,24548,https://www.zalando.ch/outfits/IMi9cMhhTSm/,high heel pumps - black,Even&Odd,https://www.zalando.ch/evenandodd-pumps-black-...,https://img01.ztat.net/article/spp-media-p1/f2...,IMi9cMhhTSm,black,woman_outfits\IMi9cMhhTSm\0.jpg,high heel pumps,black,"[high, heel, pumps]",[black],5.0
24549,24549,24549,https://www.zalando.ch/outfits/tUljaAGSTG6/,basic hooded jacket - daunenjacke - black,Tommy Jeans,https://www.zalando.ch/tommy-jeans-basic-hoode...,https://img01.ztat.net/article/spp-media-p1/0f...,tUljaAGSTG6,black,woman_outfits\tUljaAGSTG6\2.jpg,basic hooded jacket,daunenjacke,"[basic, hooded, jacket]",[daunenjacke],3.0


In [11]:
df[df['wearType'].isna()].explode('firstPartToken')['firstPartToken'].value_counts()



firstPartToken
pack        316
unisex      273
3           175
everyday    148
cush        134
           ... 
nevada        1
hairpin       1
pcillus       1
bellini       1
felicity      1
Name: count, Length: 2244, dtype: int64

In [12]:
df[df['wearType'].isna()].explode('secondPartToken')['secondPartToken'].value_counts()

secondPartToken
black                 398
hemdbluse             212
plateaustiefelette    200
sportsocken           185
strumpfhose           171
                     ... 
café                    1
forest                  1
bb                      1
mineralisches           1
bedruckt                1
Name: count, Length: 515, dtype: int64

In [13]:
df[df['wearType'].isna()].shape[0] / df.shape[0]

0.17009490448454237

In [14]:
df[df['clothingItemName'].str.contains('softshelljacke')]

,Unnamed: 0.1,Unnamed: 0,outfit_url,clothingItemName,clothingItemBrand,clothingItemDetailUrl,clothingItemPreviewImageUrl,id,clothingItemColor,image_path,fistToken,secondToken,firstPartToken,secondPartToken,wearType
34,34,34,https://www.zalando.ch/outfits/pR3dBZuDQBq/,brenham - softshelljacke - black,Icepeak,https://www.zalando.ch/icepeak-brenham-softshe...,https://img01.ztat.net/article/spp-media-p1/8a...,pR3dBZuDQBq,black,woman_outfits\pR3dBZuDQBq\2.jpg,brenham,softshelljacke,[brenham],[softshelljacke],NaN
1184,1184,1184,https://www.zalando.ch/outfits/P-AX68iLT4O/,brenham - softshelljacke - dark blue,Icepeak,https://www.zalando.ch/icepeak-brenham-softshe...,https://img01.ztat.net/article/spp-media-p1/26...,P-AX68iLT4O,dark blue,woman_outfits\P-AX68iLT4O\0.jpg,brenham,softshelljacke,[brenham],[softshelljacke],NaN
1786,1786,1786,https://www.zalando.ch/outfits/pR3dBZuDQBq/,brenham - softshelljacke - black,Icepeak,https://www.zalando.ch/icepeak-brenham-softshe...,https://img01.ztat.net/article/spp-media-p1/8a...,pR3dBZuDQBq,black,woman_outfits\pR3dBZuDQBq\14.jpg,brenham,softshelljacke,[brenham],[softshelljacke],NaN
3914,3914,3914,https://www.zalando.ch/outfits/N1PhrYmvQBW/,sangro jacket - softshelljacke - tnf black,The North Face,https://www.zalando.ch/the-north-face-sangro-h...,https://img01.ztat.net/article/spp-media-p1/3d...,N1PhrYmvQBW,tnf black,woman_outfits\N1PhrYmvQBW\0.jpg,sangro jacket,softshelljacke,"[sangro, jacket]",[softshelljacke],3.0


In [15]:
from src.clothing_item_types import WearType


def get_number_of_sets_in_dataframe(df):
    return len(list(set(df['id'])))


def set_contains_items_for_valid_outfit(set_id, df):
    items = df[df['id'] == set_id]

    has_inner_wear = False
    has_outer_wear = False
    has_shoes = False
    has_bottom_wear = False

    types = pd.DataFrame([], columns=['wearType'])

    for index, item in items.iterrows():
        item_wear_type = item["wearType"]

        if (item_wear_type == WearType.innerWear) and (not has_inner_wear):
            has_inner_wear = True
            types = pd.concat([types, pd.DataFrame([{'wearType': item_wear_type}])], ignore_index=True)

        if (item_wear_type == WearType.outerWear) and (not has_outer_wear):
            has_outer_wear = True
            types = pd.concat([types, pd.DataFrame([{'wearType': item_wear_type}])], ignore_index=True)

        if (item_wear_type == WearType.shoes) and (not has_shoes):
            has_shoes = True
            types = pd.concat([types, pd.DataFrame([{'wearType': item_wear_type}])], ignore_index=True)

        if (item_wear_type == WearType.bottomWear) and (not has_bottom_wear):
            has_bottom_wear = True
            types = pd.concat([types, pd.DataFrame([{'wearType': item_wear_type}])], ignore_index=True)

    if types.shape[0] > 0:
        counts = pd.DataFrame(types['wearType'].value_counts())
        if counts[counts['count'] > 1].shape[0] > 1:
            print('removed set')
            return False

    # if (has_outer_wear and has_shoes and has_bottom_wear) and (not has_inner_wear):
    #     return False
    #
    # if (has_inner_wear and has_shoes and has_bottom_wear) and (not has_outer_wear):
    #     return True
    # return has_inner_wear and has_inner_wear and has_shoes and has_bottom_wear

    if (has_outer_wear and has_bottom_wear) and (not has_inner_wear):
        return False

    if (has_inner_wear and has_bottom_wear) and (not has_outer_wear):
        return True

    return has_inner_wear and has_outer_wear and has_bottom_wear


set_ids_which_are_valid_outfits = []
for set_id in list(set(df['id'])):
    if set_contains_items_for_valid_outfit(set_id, df):
        set_ids_which_are_valid_outfits.append(set_id)

df_only_valid_outfits = df[df["id"].isin(set_ids_which_are_valid_outfits)]

number_of_sets_which_are_valid_outfits = get_number_of_sets_in_dataframe(df_only_valid_outfits)
print(f'number of sets which are valid outfits: {number_of_sets_which_are_valid_outfits}')

/var/folders/kn/1r08h7jx2hlcz7rndw5qc1d80000gn/T/ipykernel_32667/2431676428.py:31: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  types = pd.concat([types, pd.DataFrame([{'wearType': item_wear_type}])], ignore_index=True)
/var/folders/kn/1r08h7jx2hlcz7rndw5qc1d80000gn/T/ipykernel_32667/2431676428.py:27: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  types = pd.concat([types, pd.DataFrame([{'wearType': item_wear_type}])], ignore_index=True)
/var/folders/kn/1r08h7jx2hlcz7rndw5qc1d80000gn/T/ipykernel

number of sets which are valid outfits: 1516


/var/folders/kn/1r08h7jx2hlcz7rndw5qc1d80000gn/T/ipykernel_32667/2431676428.py:23: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  types = pd.concat([types, pd.DataFrame([{'wearType': item_wear_type}])], ignore_index=True)
/var/folders/kn/1r08h7jx2hlcz7rndw5qc1d80000gn/T/ipykernel_32667/2431676428.py:27: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  types = pd.concat([types, pd.DataFrame([{'wearType': item_wear_type}])], ignore_index=True)
/var/folders/kn/1r08h7jx2hlcz7rndw5qc1d80000gn/T/ipykernel